In [ ]:
import os  # Для работы с директориями и переменными в окружении
import re  # Для работы с регулярными выражениями

# 1 - Парсинг текста из PDF

In [ ]:
# Путь к PDF документов
folder_path_to_pdf = r"path\to\NPA"

## 1.1 - Использование PyMuPDF (fitz) для получения текста

In [ ]:
import fitz  # Для чтения текста из PDF

In [ ]:
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with fitz.open(pdf_path) as doc:
            for page_num in range(len(doc)):
                page = doc.load_page(page_num)
                text += page.get_text()
    except Exception as e:
        print(f"Ошибка извлечения текста из {pdf_path}: {e}")

    return text

## 1.2 - Определение оглавления и основного текста документа

In [ ]:
def separate_text_from_toc(text):
    # Паттерн для разделения документа
    match = re.compile(r'(.*?)(?:(?:У(?:ТВЕРЖДЕН|твержден)(?:(?:А|а)|(?:Ы|ы)))\s+приказом\s+ФСТЭК\s+России|'
                       r'Зарегистрировано\s+в\s+Министерстве\s+юстиции\s+Российской\s+Федерации|'
                       r'Одобрен\s+Советом\s+Федерации|Директор|Правительств(?:а|о))(.*)', re.DOTALL).search(text)
    if match:
        table_of_contents = match.group(1).strip()
        main_text = match.group(2).strip()
    else:
        table_of_contents = ""
        main_text = text.strip()

    return table_of_contents, main_text

## 1.3 - Очистка текста от лишней и повторяющейся информации

In [ ]:
# Словарь для преобразования русских месяцев в числовые
months = {
    'января': '01',
    'февраля': '02',
    'марта': '03',
    'апреля': '04',
    'мая': '05',
    'июня': '06',
    'июля': '07',
    'августа': '08',
    'сентября': '09',
    'октября': '10',
    'ноября': '11',
    'декабря': '12'
}

def convert_russian_date(russian_date):
    day, month, year, _ = russian_date.split()
    return f'{day.zfill(2)}.{months[month]}.{year[:-2]}'

def clean_text(text, doc_type, doc_date, doc_number):
    # Компиляция регулярного выражения для удаления идентификации документа от КонсультантПлюс
    pattern = re.compile(
        rf'{doc_type[:1].upper() + doc_type[1:].lower()} .*?{convert_russian_date(doc_date)}.*?N\s*{doc_number}\s*\(ред\.\s*от\s*\d{{2}}\.\d{{2}}\.\d{{4}}\)\s*?".*?(?:\.\.\.|")',
        re.DOTALL
    )

    # Применение регулярного выражения
    text = re.sub(pattern, '', text).strip()

    # Удаление ненужных фраз и информации от АО "Кодекс"
    text = re.split(
        r'Электронный\s+текст\s+документа\s+подготовлен\s+АО\s+"Кодекс"\s+и\s+сверен\s+по:',
        text,
        flags=re.IGNORECASE
    )[0]

    # Удаление ненужных фраз и информации от КонсультантПлюс
    text = re.sub(
        r'(Документ предоставлен|www\.consultant\.ru|КонсультантПлюс|надежная\s+правовая\s+поддержка|Страница\s*\d+\s*из\s*\d+\s|Дата сохранения:\s+\d{2}\.\d{2}\.\d{4})',
        '',
        text,
        flags=re.IGNORECASE
    ).strip()

    # Разделение текста на строки и удаление пустых строк
    lines_text = [line.strip() for line in text.split('\n') if line.strip()]

    # Удаление подчеркиваний и дефисов
    lines_text = [re.sub(r'(_+|-+)', '', line) for line in lines_text]

    # Объединение очищенных строк в одну строку
    cleaned_text = ' '.join(lines_text)
    
    return cleaned_text

## 1.4 - Извлечение идентификационных данных документа
- Оранжевое - организация;
- Фиолетовое - тип документа;
- Голубое - дата и номер документа.

<img src="figures/document_identification.jpg"/>

In [ ]:
def document_identification(text):
    # Удаляем строки, содержащие "Зарегистрировано в Минюсте России"
    text = re.sub(r'Зарегистрировано\s+в\s+Минюсте\s+России\s+\d{1,2}\s+\w+\s+\d{4}\s+г\.\s+N\s+\d+', '', text)
    
    # Паттерны для извлечения информации о документе
    document_pattern = re.compile(
        r"(?:(\d{1,2}\s+\w+\s+\d{4}\s+г(?:\.|ода)?)\s+N\s+(\d+).*(ФЗ)?|)\s*"
        r"(ФЕДЕРАЛЬНАЯ\s+СЛУЖБА\s+(?:БЕЗОПАСНОСТИ\s+РОССИЙСКОЙ\s+ФЕДЕРАЦИИ|ПО\s+ТЕХНИЧЕСКОМУ\s+И\s+ЭКСПОРТНОМУ\s+КОНТРОЛЮ)|"
        r"ПРАВИТЕЛЬСТВО\s+РОССИЙСКОЙ\s+ФЕДЕРАЦИИ|РОССИЙСКАЯ\s+ФЕДЕРАЦИЯ|ПРЕЗИДЕНТ\s+РОССИЙСКОЙ\s+ФЕДЕРАЦИИ)\s*"
        r"((?:ПРИКАЗ|ПОСТАНОВЛЕНИЕ|УКАЗ|ФЕДЕРАЛЬНЫЙ\s+ЗАКОН))(?:\s+от\s+(\d{1,2}\s+\w+\s+\d{4}\s+г(?:\.|ода)?)\s+(?:N|№)\s+(\d+)|)"
    )

    # Примеры:
    #   ФЕДЕРАЛЬНАЯ СЛУЖБА БЕЗОПАСНОСТИ РОССИЙСКОЙ ФЕДЕРАЦИИ ПРИКАЗ от 6 мая 2019 года N 196 Об утверждении Требований к средс...
    #   ПРАВИТЕЛЬСТВО РОССИЙСКОЙ ФЕДЕРАЦИИ ПОСТАНОВЛЕНИЕ от 8 февраля 2018 г. № 127 МОСКВА Об утверждении Правил категорирован...
    #   26 июля 2017 года N 187ФЗ РОССИЙСКАЯ ФЕДЕРАЦИЯ ФЕДЕРАЛЬНЫЙ ЗАКОН О БЕЗОПАСНОСТИ КРИТИЧЕСКОЙ ИНФОРМАЦИОННОЙ ИНФРАСТРУКТ...
    #   ПРЕЗИДЕНТ РОССИЙСКОЙ ФЕДЕРАЦИИ УКАЗ от 30 марта 2022 г. N 166 О МЕРАХ ПО ОБЕСПЕЧЕНИЮ ТЕХНОЛОГИЧЕСКОЙ НЕЗАВИСИМОСТИ И Б...

    # Ищем соответствия в тексте
    document_match = document_pattern.search(text)

    if document_match:
        if document_match.group(1) is None:
            organization = document_match.group(4)
            document_type = document_match.group(5)
            date = document_match.group(6)
            document_number = document_match.group(7)
        else:
            organization = document_match.group(4)
            document_type = document_match.group(5)
            date = document_match.group(1)
            document_number = document_match.group(2)

        return {
            'organization': organization,
            'document_type': document_type,
            'date': date,
            'document_number': document_number
        }
    else:
        print("Не удалось извлечь информацию из текста")
        return None

## 1.5 - Извлечение заголовков и (под)пунктов
Используется счетчик для корректировки записи распознанного массива (под)пунктов.

In [ ]:
def extract_titles_and_texts(text):
    """
    Извлекает заголовки разделов с римскими цифрами, "Статья X" и заголовки по шаблону "Приложение N X".
    """
    roman_pattern = re.compile(r"(?<=\b)([IVXLCDM]+\.\s+.+?)(?=\s+[IVXLCDM]+\.\s|\s*\d+\.\s|$)", re.MULTILINE | re.DOTALL)
    article_pattern = re.compile(r"(?<=\b)(Статья\s+\d+\.\s+.+?)(?=\s+Статья\s+\d+\.\s|\s*\d+(?:\.|\))\s|$)", re.MULTILINE | re.DOTALL)
    appendix_order_pattern = re.compile(r"(?<=\b)(Приложение\s+N\s+\d+\s+к\s+приказу\s+ФСБ\s+России\s+от\s+\d+\s+\w+\s+\d{4}\s+г(?:\.|ода)\s+N\s+\d+)(?=\s+)", re.MULTILINE | re.DOTALL)

    matches = list(re.finditer(roman_pattern, text)) + list(re.finditer(article_pattern, text)) + list(re.finditer(appendix_order_pattern, text))
    matches.sort(key=lambda x: x.start())

    results = []
    for i, match in enumerate(matches):
        title = match.group(1).strip()
        start_pos = match.end()

        end_pos = matches[i + 1].start() if i + 1 < len(matches) else len(text)
        results.append((title, text[start_pos:end_pos].strip()))

    return results

def get_item_numbers(text):
    """
    Извлекает номера пунктов из текста.
    """
    pattern = re.compile(r"((?:\d*\.)*)\s*")
    match = pattern.match(text)
    if match:
        return match.group(1).strip().rstrip('.').split('.')
    
    return []

def extract_subpoints(text, counter_status=['0'], register_counter_status=0):
    """
    Извлекает подпункты из текста и проверяет их счетчиком.
    """
    
    def update_counters(subpoint_number, counter_status, register_counter_status):
        """
        Обновляет счетчики для подуровней разделов.
        """
        if len(subpoint_number) > len(counter_status):
            register_counter_status += 1
            counter_status.append('0')
        elif len(subpoint_number) < len(counter_status):
            register_counter_status -= 1
            counter_status.pop()

        counter_status[register_counter_status] = str(int(counter_status[register_counter_status]) + 1)

        return counter_status, register_counter_status

    subpoint_pattern = re.compile(r"((?:\d+\.)+\s*.+?)(?=(?:\d+\.)+|$)", re.DOTALL)
    matches = re.findall(subpoint_pattern, text)
    subpoints = []

    first_register_occupied = False
    
    for match in matches:
        subpoint = match.strip()
        subpoint_number = get_item_numbers(subpoint)
        if subpoint_number:
            counter_status, register_counter_status = update_counters(subpoint_number, counter_status, register_counter_status)

            # Логирование
            # print(f"Счетчик: {counter_status}, Распознано: {subpoint_number}, Уровень подпункта: {register_counter_status}")

            if not first_register_occupied:
                if subpoint_number == counter_status:
                    first_register_occupied = True
                else:
                    counter_status[register_counter_status] = str(int(counter_status[register_counter_status]) - 1)
            if first_register_occupied:
                if subpoint_number != counter_status:
                    if subpoints:
                        subpoints[-1] += ' ' + subpoint
                        counter_status[register_counter_status] = str(int(counter_status[register_counter_status]) - 1)
                else:
                    subpoints.append(subpoint)
        
    return subpoints, counter_status, register_counter_status

## 1.6 - Получение информации из PDF

In [ ]:
def process_pdf_file(text, doc_info):
    """
    Обрабатывает PDF файл: извлекает текст, разделяет его на оглавление и основной текст,
    идентифицирует документ, извлекает заголовки и их подпункты.
    """
    titles_and_subpoints = {}

    # Поиск заголовков
    titles = extract_titles_and_texts(text)

    # Инициализация счетчиков подпунктов
    counter_subpoints = ['0']
    register_counter_status = 0

    len_text_render = 50
    if titles:
        for title, content in titles:
            if re.match(r"Статья\s+\d+\.", title) or re.match(r"Приложение\s+N\s+\d+", title):
                # Обнуляем счетчики
                counter_subpoints = ['0']
                register_counter_status = 0

                # Извлекаем нужный текст заголовка
                if re.match(r"Приложение\s+N\s+\d+", title):
                    title = re.match(r"Приложение\s+N\s+\d+", title).group()
            
            print(f"\t({len(title)}) {title}")
            subpoints, counter_subpoints, register_counter_status = extract_subpoints(content, counter_subpoints, register_counter_status)
            titles_and_subpoints[title] = subpoints
            for subpoint in subpoints:
                print(f"\t\t({len(subpoint)}) {subpoint[:len_text_render]} [..]")
    else:
        subpoints, counter_subpoints, register_counter_status = extract_subpoints(text, counter_subpoints, register_counter_status)
        for subpoint in subpoints:
            print(f"\t({len(subpoint)}) {subpoint[:len_text_render]} [..]")
        titles_and_subpoints["Без заголовка"] = subpoints

    return {
        'doc_info': doc_info,
        'titles_and_subpoints': titles_and_subpoints,
        'text': text
    }

def extract_info_from_pdfs(folder_path):
    """
    Извлекает информацию из всех PDF файлов в указанной папке.
    """
    pdf_texts = {}
    for filename in os.listdir(folder_path):
        # Фильтр для вывода хорошо предобработанных докуменатов:
        # if filename.endswith(".pdf") and ('FSB' in filename or 'FSTEK' in filename) and 'FSTEK239' not in filename and 'FSTEK75' not in filename
        if filename.endswith(".pdf") and ('FSB' in filename or 'FSTEK' in filename) and 'FSTEK239' not in filename and 'FSTEK75' not in filename:
            pdf_file = os.path.join(folder_path, filename)
            print(filename)
            text = extract_text_from_pdf(pdf_file)
            if text:
                doc_info, text = separate_text_from_toc(text)
                doc_info = ' '.join([line.strip() for line in doc_info.split('\n') if line.strip()])
                doc_info = document_identification(doc_info)
                print(f"Орг: {doc_info['organization']}\nТип: {doc_info['document_type']}\nДата: {doc_info['date']}\nНомер: {doc_info['document_number']}\n")

                text = clean_text(text, doc_info['document_type'], doc_info['date'], doc_info['document_number'])
                if text:
                    pdf_data = process_pdf_file(text, doc_info)
                    pdf_texts[filename] = {
                        'filename': filename,
                        **pdf_data
                    }
                    print('-' * 100)
            
    return pdf_texts

pdf_texts = extract_info_from_pdfs(folder_path_to_pdf)

# 2 - Подготовка, создание графа

## 2.1 - Подготовка

### 2.1.1 - Разделение текста на предложения
Для разделения большого текста на предложения используется функция `sent_tokenize` и набор пунктуации из библиотеки `nltk`.

Если весь текст это предложение, то он будет разбит на чанки по знакам пунктуации.

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize

In [ ]:
# Набор пунктуации nltk
nltk.download('punkt')

In [ ]:
def split_text_by_punctuation(text, max_chunk_size):
    """
    Разделяет текст примерно в середине, с учетом размера чанка и пунктуации.
    """
    def find_split_index(text, max_chunk_size):
        # Найти индекс для разрыва текста около середины, но не превышая max_chunk_size
        mid = len(text) // 2
        # Найти ближайшую пунктуацию к середине текста
        split_points = re.finditer(r'[.,:;!?]\s*', text)
        best_split = None
        for point in split_points:
            if point.start() <= mid <= point.end():
                best_split = point.end()
                break
            if point.start() < mid:
                best_split = point.end()
            else:
                break
            
        return best_split if best_split and best_split <= max_chunk_size else mid

    chunks = []
    while len(text) > max_chunk_size:
        split_index = find_split_index(text, max_chunk_size)
        chunks.append(text[:split_index].strip())
        text = text[split_index:].strip()
    
    chunks.append(text)

    return chunks

### 2.1.2 - Суммаризация (Модель IlyaGusev/mbart_ru_sum_gazeta)
Если длина предложения превысит размер чанка, то из текста предложения достаётся смысловая выжимка.

Пример использования:
 - Пусть `article_text` это текст для обработки.

`input_ids = tokenizer([article_text], max_length=600, truncation=True, return_tensors="pt")["input_ids"].to("cuda")`

`output_ids = model.generate(input_ids=input_ids, no_repeat_ngram_size=4)[0]`

`summary = tokenizer.decode(output_ids, skip_special_tokens=True)`

In [ ]:
# Для инициализации модели для суммаризации текста
from transformers import MBartTokenizer, MBartForConditionalGeneration

In [ ]:
summary_model_name = "IlyaGusev/mbart_ru_sum_gazeta"

summary_tokenizer = MBartTokenizer.from_pretrained(summary_model_name)  # Скачивание модели
summary_model = MBartForConditionalGeneration.from_pretrained(summary_model_name)  # Загрузка модели
summary_model.to("cuda")

In [ ]:
def summary_text(text, chunk_size, summary_tokenizer, summary_model):
    """
    Сжимает текст до размера чанка, используя модель суммаризации.
    """
    len_summary = chunk_size + 1
    previous_len = len(text)
    no_change_count = 0

    while len_summary > chunk_size and no_change_count < 3:
        input_ids = summary_tokenizer(
            [text],
            max_length=600,
            truncation=True,
            return_tensors="pt",
        )["input_ids"].to("cuda")
        output_ids = summary_model.generate(
            input_ids=input_ids,
            no_repeat_ngram_size=4
        )[0]
        new_text = summary_tokenizer.decode(output_ids, skip_special_tokens=True)
        
        if len(new_text) >= previous_len:
            no_change_count += 1
        else:
            no_change_count = 0
        
        text = new_text
        len_summary = len(text)
        previous_len = len(text)
    
    return text

### 2.1.3 - Доступ к LLM через API от HuggingFace (mistralai/Mixtral-8x7B-Instruct-v0.1)
Инициализация LLMs для дальнейшего использования этого в функции, которая конвертирует текст в узлы и отношения между ними.

Нужен [API ключ](https://huggingface.co/settings/tokens) в режиме `WRITE`.

Выбранные модели:
1) mistralai/Mistral-7B-Instruct-v0.1
2) mistralai/Mistral-7B-Instruct-v0.2
3) mistralai/Mistral-7B-Instruct-v0.3
4) mistralai/Mixtral-8x7B-Instruct-v0.1
5) meta-llama/Meta-Llama-3-8B-Instruct

In [ ]:
from getpass import getpass  # Для хранения секретных ключей

# Для инициализации и работы с LLM
from langchain_community.llms import HuggingFaceEndpoint
from langchain_experimental.graph_transformers import LLMGraphTransformer

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass(prompt="Введите API ключ от HuggingFaceHub")

In [ ]:
models = [
    "mistralai/Mistral-7B-Instruct-v0.1",
    "mistralai/Mistral-7B-Instruct-v0.2",
    "mistralai/Mistral-7B-Instruct-v0.3",
    "mistralai/Mixtral-8x7B-Instruct-v0.1",
    "meta-llama/Meta-Llama-3-8B-Instruct"
]

def initialize_endpoint(model_id):
    return HuggingFaceEndpoint(repo_id=model_id)

def initialize_transformer(model_id):
    hg_llm = HuggingFaceEndpoint(repo_id=model_id)
    return LLMGraphTransformer(llm=hg_llm)

### 2.1.4 - Работа с Neo4j в AuraDB
Граф собирается при использовании Neo4j через облачную версию [AuraDB](https://neo4j.com/cloud/platform/aura-graph-database/).

In [ ]:
from neo4j import GraphDatabase  # Для подключения к среде Neo4j

In [ ]:
# Данные для входа в среду Neo4j
os.environ["NEO4J_URI"] = "neo4j+s://..."
os.environ["NEO4J_USERNAME"] = "..."
os.environ["NEO4J_PASSWORD"] = "..."

In [ ]:
class InteractionNeo4jGraph:
    def __init__(self):
        self.driver = GraphDatabase.driver(
            os.getenv("NEO4J_URI"),
            auth=(os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))
        )

    def close(self):
        """
        Завершает работу с Neo4j.
        """
        self.driver.close()

    def create_node(self, node_text, node_type, properties={}):
        """
        Добавляет узел, независимо от существующих в среде.
        Если узел с похожими параметрами уже есть, то создается клон с другим <id>.
        """
        with self.driver.session() as session:
            result = session.run(f"""
                CREATE (n:{node_type} {{text: $node_text}})
                SET n += $properties
                RETURN elementId(n) AS node_id
            """, node_text=node_text, properties=properties)
            return result.single()["node_id"]

    def addition_node(self, node_text, node_type, properties={}):
        """
        Добавляет узел, в зависимости от существующих в среде.
        Если узел с похожими параметрами уже есть, то функция обновит их на новые.
        """
        with self.driver.session() as session:
            result = session.run(f"""
                MERGE (n:{node_type} {{text: $node_text}})
                SET n += $properties
                RETURN elementId(n) AS node_id
            """, node_text=node_text, properties=properties)
            return result.single()["node_id"]

    def add_relationship(self, source_node_id, target_node_id, rel_type):
        """
        Добавляет отношения между узлами.
        """
        with self.driver.session() as session:
            session.run(f"""
                MATCH (a) WHERE elementId(a) = $source_node_id
                MATCH (b) WHERE elementId(b) = $target_node_id
                MERGE (a)-[r:{rel_type}]->(b)
            """, source_node_id=source_node_id, target_node_id=target_node_id)

    def find_nodes_by_text(self, node_text):
        """
        Поиск узлов по полностью указанному тексту в них.
        """
        with self.driver.session() as session:
            result = session.run(f"""
                MATCH (n {{text: $node_text}})
                RETURN n
            """, node_text=node_text)
            return [record["n"] for record in result]

    def find_similar_nodes_by_text(self, node_text):
        """
        Поиск узлов по частично указанному тексту в них.
        """
        def split_and_trim(text):
            """
            Функция для разбивки текста на части и удаления половины символов у каждой части.
            """
            parts = text.split()
            trimmed_parts = [part[:len(part)//2] for part in parts]
            return trimmed_parts

        # Разбиваем текст и удаляем половину символов у каждой части
        trimmed_texts = split_and_trim(node_text)

        # Формируем запрос
        query = "MATCH (n) WHERE "
        params = {}
        # Создаем условия для поиска узлов по каждому обрезанному тексту
        for i, trimmed_text in enumerate(trimmed_texts):
            query += f"n.text CONTAINS $node_text{i}"
            if i < len(trimmed_texts) - 1:
                query += " AND "
            params[f"node_text{i}"] = trimmed_text
        query += " RETURN elementId(n) AS node_id"

        # Отправляем запрос
        with self.driver.session() as session:
            result = session.run(query, **params)
            return [record["node_id"] for record in result]

    def find_node_id_by_text(self, node_text):
        """
        Поиск ID узла по тексту.
        """
        with self.driver.session() as session:
            result = session.run(f"""
                MATCH (n {{text: $node_text}})
                RETURN elementId(n) AS node_id
            """, node_text=node_text)
            node = result.single()
            return node["node_id"] if node else None
    
    def find_types_node_by_id(self, node_id):
        """
        Получение состояния поля "типы" узла по ID. 
        """
        with self.driver.session() as session:
            result = session.run(f"""
                MATCH (n) WHERE elementId(n) = $node_id
                RETURN n.типы AS node_types
            """, node_id=node_id)
            record = result.single()
            return record["node_types"] if record else None
    
    def updating_types_node_by_id(self, node_id, properties_types):
        """
        Обновление состояния поля "типы" узла по ID.
        """
        with self.driver.session() as session:
            session.run(f"""
                MATCH (n) WHERE elementId(n) = $node_id
                SET n.типы = $properties_types
            """, node_id=node_id, properties_types=properties_types)
    
    def find_text_node_by_id(self, node_id):
        """
        Получение состояния поля "text" узла по ID. 
        """
        with self.driver.session() as session:
            result = session.run(f"""
                MATCH (n) WHERE elementId(n) = $node_id
                RETURN n.text AS node_text
            """, node_id=node_id)
            record = result.single()
            return record["node_text"] if record else None
    
    def check_current_node_type_by_id(self, node_id):
        """
        Получение типа узла по ID. 
        """
        with self.driver.session() as session:
            result = session.run(f"""
                MATCH (n) WHERE elementId(n) = $node_id
                RETURN labels(n) AS node_type
            """, node_id=node_id)
            record = result.single()
            return record["node_type"] if record else None

### 2.1.5 - Перевод текста на русский

In [ ]:
from deep_translator import GoogleTranslator  # Для перевода текста на русский

In [ ]:
def translate_to_rus(text):
    # Проверяем, содержит ли текст английские буквы
    if re.search(r'[a-zA-Z]', text):
        return GoogleTranslator(source='en', target='ru').translate(text)
    else:
        return text
def translate_to_en(text):
    # Проверяем, содержит ли текст английские буквы
    if re.search(r'[а-яёА-ЯЁ]', text):
        return GoogleTranslator(source='ru', target='en').translate(text)
    else:
        return text

### 2.1.6 - Подготовка ID и типа узла перед добавлением 

In [ ]:
def prepare_text_node(text):
    """
    Подготовка текста для вставки в узел.

    Пример return: "Тест текст"
    """
    
    # Проверка, является ли текст заголовком
    if re.match(r'^[IVXLCDM]+\.\s+', text, re.IGNORECASE) or \
       re.match(r"Статья\s+\d+\.", text, re.IGNORECASE) or \
       re.match(r"Приложение\s+N\s+\d+", text, re.IGNORECASE):
        return text

    # Превращение возможных разделяющих знаков в пробелы
    text = re.sub(r'[-_]', ' ', text)
    # Удаление лишних пробелов и проверка наличия текста
    text = text.strip()
    if not text:
        return None
    # Вставка пробелов между маленькими и большими буквами
    text = re.sub(r'([a-zа-яё])([A-ZА-ЯЁ])', r'\1 \2', text).strip()
    # Попытка перевести текст
    text = translate_to_rus(text)
    # Форматирование аббревиатуры
    text = re.sub(r'(?:Gos SOPKA|Гос СОПКА|Госсопка|Гос сопка)', 'ГосСОПКА', text)

    # Разделение текста на слова
    words = text.split(' ')
    formatted_words = []
    
    # Обработка первого слова
    if words:
        if re.search(r'[А-ЯЁA-Z]{2,}', words[0]):
            first_word = words[0]
        else:
            first_word = words[0].capitalize()
        formatted_words.append(first_word)
    
    # Обработка оставшихся слов
    for word in words[1:]:
        if re.search(r'[А-ЯЁA-Z]{2,}', word):
            formatted_words.append(word)
        else:
            formatted_words.append(word.lower())

    return ' '.join(formatted_words)

def prepare_type_node(text):
    """
    Подготовка названия типа узла.

    Пример return: "ТестТекст"
    """

    # Проверка на принадлежность к собственному набору
    if re.match(r'(?:Организация|НомерДокумента|Заголовок|Пункт)', text):
        return text

    # Превращение возможных разделяющих знаков в пробелы
    text = re.sub(r'[-_]', ' ', text)
    # Удаление лишних пробелов и проверка наличия текста
    text = text.strip()
    if not text:
        return None
    # Вставка пробелов между маленькими и большими буквами
    text = re.sub(r'([a-zа-яё])([A-ZА-ЯЁ])', r'\1 \2', text).strip()
    # Попытка перевести текст
    text = translate_to_rus(text).replace('-', ' ')
    # Приведение к общему виду
    text = text.capitalize()

    # Разделение текста на слова, приведение каждого слова к нижнему регистру, кроме первого символа
    text = ''.join([word.capitalize() for word in text.split()])

    return text

### 2.1.7 - Лемматизация слов
Для поиска похожих узлов и исключения случаев добавление по сути одинаковых узлов, но отличающихся только их окончанием.

In [ ]:
from pymorphy2 import MorphAnalyzer  # Для морфологического анализа слов

In [ ]:
# Инициализация морфологического анализатора
morph = MorphAnalyzer()

def lemmatize_word(word):
    # Приведение слова к начальной форме
    p = morph.parse(word)[0]
    return p.normal_form

def lemmatize_text(text):
    # Разделение текста на слова и лемматизация каждого слова
    words = text.split()
    lemmatized_words = [lemmatize_word(word) for word in words]
    lemmatized_words = ' '.join(lemmatized_words)
    return lemmatized_words[:1].upper() + lemmatized_words[1:]

## 2.2 - Создание RAG графа

In [ ]:
from langchain_core.documents import Document  # Для конвертации текста в допустимый формат функции "convert_to_graph_documents"
import json_repair  # Для работы функции "convert_to_graph_documents"

import json  # Для логирования успешно завершенных этапов

In [ ]:
# Длинна чанка текста поступающего в функцию "convert_to_graph_documents"
chunk_size = 500

### 2.2.1 - Логирование
Это поможет избежать дублирования узлов при перезапуске ноутбука.

In [ ]:
# Загружаем существующий лог или создаём новый
log_file = 'processed_files_log.json'
if os.path.exists(log_file):
    with open(log_file, 'r', encoding='utf-8') as file:
        processed_log = json.load(file)
else:
    processed_log = {}

# Функция для обновления лога
def update_log(log_file, filename, title, title_id_node, paragraph, paragraph_id_node, processed_log, status):
    if filename not in processed_log:
        processed_log[filename] = {}
        
    if title not in processed_log[filename]:
        processed_log[filename][title] = []

    # Найти запись
    entry_title_id_node = next((item for item in processed_log[filename][title] if item['title_id_node'] == title_id_node), None)
    if entry_title_id_node:
        if paragraph not in entry_title_id_node:
            entry_title_id_node[paragraph] = {}
        entry_title_id_node[paragraph]["paragraph_id_node"] = paragraph_id_node
        entry_title_id_node[paragraph]["status"] = status
    else:
        processed_log[filename][title].append({
            "title_id_node": title_id_node,
            paragraph: {"paragraph_id_node": paragraph_id_node,
                        "status": status}
        })

    with open(log_file, 'w', encoding='utf-8') as file:
        json.dump(processed_log, file, ensure_ascii=False, indent=4)

### 2.2.2 - Получение графа от LLM и заполнение среды Neo4j

In [ ]:
# Функция для заполнения графа
def filling_in_graph(document, paragraph_id_node, graph, models, filter=True):
    global hg_llm_transformer
    all_models_failed = True
    stop = False
    
    for model_id in models:
        try:

            # Использование LLM для построения графа:
            
            if filter:
                print("Использование фильтра..")
                #   Фильрация:
                # Инициализация LLM
                hg_llm_endpoint = initialize_endpoint(model_id)
                # Фильтрация параметров
                hg_llm_transformer = LLMGraphTransformer(
                    llm=hg_llm_endpoint,
                    allowed_nodes=[
                        "Document", "Organization", "Person", "Group", "Information", 
                        "Legislation", "System", "Security", "Process", 
                        "Location", "Event", "Resources", "PoliticalParty", "Incident", "Decision"
                    ],
                    allowed_relationships=[
                        "IN_ACCORDANCE_WITH", "INTERACTS_WITH", "OWNS", "USES",
                        "LOCATED_IN", "RELATED_TO", "ENSURES", "CONNECTED_TO", "HAS_TYPE",
                        "RESPONSIBLE_FOR", "CONTAINS", "REGULATES", "BELONGS_TO", "IS",
                        "PROVIDES"
                    ]
                )
                graph_documents = hg_llm_transformer.convert_to_graph_documents(document)
            else:
                print("Обход фильтра..")
                #   Без фильтрации:
                # Инициализация LLM
                hg_llm_transformer = initialize_transformer(model_id)
                graph_documents = hg_llm_transformer.convert_to_graph_documents(document)
                stop = True

            # Вывод результатов
            print(f"\n({len(graph_documents[0].nodes)}) Nodes: {graph_documents[0].nodes}")
            print(f"({len(graph_documents[0].relationships)}) Relationships: {graph_documents[0].relationships}")
            
            # Событие когда LLM выводит пустой граф
            if len(graph_documents[0].nodes) == 0:
                print(f"\n{model_id} выдала пустой граф.\n")
                continue
            all_models_failed = False
            
            # Создание словаря для сохранения ID по тексту в нём
            individual_dictionary_nodes_ids = {}
            
            # Заполнение графа в Neo4j
            for graph_document in graph_documents:
                # Заполнение в Neo4j распознанных сущностей
                for node in graph_document.nodes:
                    # Подготовка данных
                    prepare_text = prepare_text_node(node.id)
                    prepare_type = prepare_type_node(node.type)
                    
                    # Исключение добавления None сущностей
                    if prepare_text is None or prepare_type is None:
                        continue
                    
                    # Проверка на существование такого узла
                    find_node_id = graph.find_node_id_by_text(prepare_text)
                    individual_dictionary_nodes_ids[prepare_text] = None
                    if find_node_id == None:
                        # Попытка найти узел с похожим текстом
                        similar_nodes_ids = graph.find_similar_nodes_by_text(prepare_text)
                        if similar_nodes_ids != []:
                            print(f"Обнаружено возможное совпадение в узлах: `{similar_nodes_ids}`")
                            for similar_node_id in similar_nodes_ids:
                                # Проверка на одинаковое содержание
                                print(f"Лемма текущего текста: `{lemmatize_text(prepare_text)}`; Лемма найденного текста: `{lemmatize_text(graph.find_text_node_by_id(similar_node_id))}`")
                                if lemmatize_text(prepare_text) == lemmatize_text(graph.find_text_node_by_id(similar_node_id)):
                                    # Если найден узел с очень похожим содержанием, то
                                    # создаём отношение: "Пункт" -> "X-старая-сущность"
                                    print(f"Обнаружен похожий текст: `{prepare_text}`; В узле c ID: `{similar_node_id}`")
                                    graph.add_relationship(source_node_id=paragraph_id_node,
                                                   target_node_id=similar_node_id,
                                                   rel_type='СОДЕРЖАНИЕ')
                                    # Сохранение ID узла
                                    individual_dictionary_nodes_ids[prepare_text] = similar_node_id
                                    break # Остановка поиска

                    # Если такой узел существует то обновляем в уже существующем узле поле "типы"    
                    else:
                        # Поиск уже существующих типов узла
                        find_types = graph.find_types_node_by_id(find_node_id)
                        if find_types:
                            find_types = find_types.split('; ')
                        else:
                            find_types = []
                        # Добавление нового типа
                        # Исключение добавления присвоенного типа
                        current_node_type = graph.check_current_node_type_by_id(find_node_id)[0]
                        if prepare_type not in find_types and current_node_type != prepare_type:
                            find_types.append(prepare_type)
                            find_types = '; '.join(find_types)
                            graph.updating_types_node_by_id(node_id=find_node_id, properties_types=find_types)
                        
                        # Если найден узел с похожим содержанием, то
                        # создаём отношение: "Пункт" -> "X-старая-сущность"
                        print(f"Обнаружен похожий текст: `{prepare_text}`; В узле c ID: `{find_node_id}`")
                        graph.add_relationship(source_node_id=paragraph_id_node,
                                       target_node_id=find_node_id,
                                       rel_type='СОДЕРЖАНИЕ')
                        # Сохранение ID узла
                        individual_dictionary_nodes_ids[prepare_text] = find_node_id
                    
                    # Если такого узла не нашлось, то он создаётся
                    if individual_dictionary_nodes_ids[prepare_text] == None:
                        # Создание отношения: "Пункт" -> "X-новая-сущность"
                        id_node = graph.addition_node(node_text=prepare_text, node_type=prepare_type)
                        print(f"Создан новый узел c ID: `{id_node}`; Текст: `{prepare_text}`")
                        graph.add_relationship(source_node_id=paragraph_id_node,
                                               target_node_id=id_node,
                                               rel_type='СОДЕРЖАНИЕ')
                        # Сохранение ID узла
                        individual_dictionary_nodes_ids[prepare_text] = id_node
                
                # Заполнение в Neo4j распознанных отношений между сущностями
                for relationship in graph_document.relationships:
                    # Подготовка текста для поиска ID узлов
                    prepare_source_node_text = prepare_text_node(relationship.source.id)
                    prepare_target_node_text = prepare_text_node(relationship.target.id)

                    # Исключение добавления отношений между None сущностями
                    if prepare_source_node_text is None or prepare_target_node_text is None:
                        continue

                    # Поиск ID узлов в словаре
                    relationship_source_id = None
                    relationship_target_id = None
                    if prepare_source_node_text in individual_dictionary_nodes_ids \
                       and prepare_target_node_text in individual_dictionary_nodes_ids:
                        relationship_source_id = individual_dictionary_nodes_ids[prepare_source_node_text]
                        relationship_target_id = individual_dictionary_nodes_ids[prepare_target_node_text]
                    
                    print(f"{relationship_source_id} -> {relationship_target_id}")

                    if relationship_source_id is None \
                       or relationship_target_id is None:
                        print("Данных об отношении не было найдено:")
                        print(f"\tСловарь: {individual_dictionary_nodes_ids}")
                        print(f"\tИсточник: `{prepare_source_node_text}`; Цель: `{prepare_target_node_text}`")

                    # Определение названия отношения
                    relationship_type = translate_to_rus(relationship.type.replace('_', ' ').replace('-', ' ')).upper().replace(' ', '_').replace('-', '_')

                    # Исключение добавления None отношений и отношений на один и тот же узел
                    if relationship_source_id is not None \
                       and relationship_target_id is not None \
                       and relationship_source_id != relationship_target_id:

                        graph.add_relationship(source_node_id=relationship_source_id,
                                               target_node_id=relationship_target_id,
                                               rel_type=relationship_type)
            
            print('-' * 100)
            # break  # Выход из цикла при первом результате
        except Exception as e:
            print(f"\nОшибка при использовании модели {model_id}: {e}\n")
            continue
    
    if all_models_failed:
        print("\nВсе модели выдали плохой результат.")
        if not stop:
            print("\n\tПерезапуск анализа чанка, но без фильтра...")
            result = filling_in_graph(document, paragraph_id_node, graph, models, filter=False)
        if stop:
            print("\nМодели продолжают выдавать плохой результат. Пропуск чанка.")
            print('-' * 100)
            return False
        return result
    return True

### 2.2.3 - Запуск анализа выбранных документов

In [ ]:
# Инициализируем граф
graph = InteractionNeo4jGraph()


# Основной цикл обработки файлов
for filename in pdf_texts:
    document_info = pdf_texts[filename]['doc_info']

    # Пропуск не идентифицированных документов
    if document_info is None:
        continue
    
    # Идентификация документа
    prepare_org_name = prepare_text_node(document_info['organization'])
    prepare_doc_number = prepare_text_node(document_info['document_number'])
    doc_type = document_info['document_type']
    doc_date = document_info['date']

    # Создание отношения: "Организация" -> "Документ"
    org_id_node = graph.addition_node(node_text=prepare_org_name, node_type=prepare_type_node("Организация"))
    doc_info_id_node = graph.addition_node(node_text=prepare_doc_number, node_type=prepare_type_node("НомерДокумента"), properties={"тип": doc_type,"дата": doc_date})
    graph.add_relationship(source_node_id=org_id_node,
                           target_node_id=doc_info_id_node,
                           rel_type="ИЗДАННО")
    
    # Чтение информации о созданных узлах документа из лог-файла
    log_file_data = processed_log.get(filename, None)

    # Привязывание распознаваемых сущностей к заголовкам и пунктам
    for titles_and_subpoints in pdf_texts[filename]['titles_and_subpoints']:

        # Если документ имеет хотя бы один заголовок, то создаётся отношение:
        #   "Документ" -> "Заголовок"
        title_id_node = None
        log_title_data = None
        prepare_title = prepare_text_node(titles_and_subpoints)
        if log_file_data:
            # Поиск в лог-файле ID узла "Заголовок"
            log_title_data = log_file_data.get(prepare_title, None)
        if prepare_title != 'Без заголовка':
            if log_title_data:
                # Если в лог-файле обнаружен ID узла "Заголовок", то используем его
                title_id_node = log_title_data[0].get("title_id_node", None)
            if title_id_node:
                print(f"Узел `Заголовок`: `{prepare_title}` уже существует; ID: {title_id_node}")
            # Иначе узел "Заголовок" создаётся
            else:
                title_id_node = graph.create_node(node_text=prepare_title, node_type=prepare_type_node('Заголовок'))
                graph.add_relationship(source_node_id=doc_info_id_node,
                                       target_node_id=title_id_node,
                                       rel_type='ЗАГОЛОВОК')

        # Обработка (под)пунктов
        for text in pdf_texts[filename]['titles_and_subpoints'][titles_and_subpoints]:
            
            # Если документ имеет хотя бы один заголовок, то создаётся отношение:
            #   "Заголовок" -> "Пункт"
            prepare_paragraph = prepare_text_node(f"{'.'.join(get_item_numbers(text))}.")
            paragraph_id_node = None
            if log_file_data and log_title_data:
                # Поиск в лог-файле ID узла "Пункт"
                paragraph_data = log_title_data[0].get(prepare_paragraph, {})
            if titles_and_subpoints != 'Без заголовка':
                if log_file_data and paragraph_data:
                    # Если в лог-файле обнаружен ID узла "Пункт", то используем его
                    paragraph_id_node = paragraph_data.get("paragraph_id_node", None)
                # Если в лог-файле обнаружен ID узла "Пункт", то используем его
                if paragraph_id_node:
                    print(f"Узел `Пункт`: `{prepare_paragraph}` уже существует; ID: {paragraph_id_node}")
                    # Если в лог-файле обнаружено, что обработка не закончена, то продолжаем
                    status_paragraph = paragraph_data.get("status", None)
                    if status_paragraph == 'completed':
                        continue
                # Иначе узел "Пункт" создаётся
                else:
                    paragraph_id_node = graph.create_node(node_text=prepare_paragraph, node_type=prepare_type_node('Пункт'))
                    graph.add_relationship(source_node_id=title_id_node,
                                           target_node_id=paragraph_id_node,
                                           rel_type='СОДЕРЖАНИЕ')
            
            # Иначе узел "Заголовок" в цепочке пропускается. Остаётся:
            #   "Документ" -> "Пункт"
            else:
                if log_file_data:
                    # Поиск в лог-файле ID узла "Пункт"
                    paragraph_id_node = paragraph_data.get("paragraph_id_node", None)
                # Если в лог-файле обнаружен ID узла "Пункт", то используем его
                if log_file_data and paragraph_id_node:
                    print(f"Узел `Пункт`: `{prepare_paragraph}` уже существует; ID: {paragraph_id_node}")
                    # Если в лог-файле обнаружено, что обработка не закончена, то продолжаем
                    status_paragraph = paragraph_data.get("status", None)
                    if status_paragraph == 'completed':
                        continue
                # Иначе узел "Пункт" создаётся
                else:
                    paragraph_id_node = graph.create_node(node_text=prepare_paragraph, node_type=prepare_type_node('Пункт'))
                    graph.add_relationship(source_node_id=doc_info_id_node,
                                           target_node_id=paragraph_id_node,
                                           rel_type='СОДЕРЖАНИЕ')

            # Отчет о том что узлы уже созданы в графе, нужно найти сущности и отношения
            update_log(log_file, filename, prepare_title, title_id_node, prepare_paragraph, paragraph_id_node, processed_log, 'in progress')

            # Удаление их текста номера пункта
            text = text[len(prepare_paragraph):]

            # Суммаризация текста до заданной длины чанка
            if len(text) > chunk_size:
                text = summary_text(text, chunk_size, summary_tokenizer, summary_model)
                # print(text)

            # Принятие мер, если суммаризированный текст слишком велик
            if len(text) > chunk_size:
                # Разделение текста на предложения
                text = sent_tokenize(text)
                # Если текст это одно предложение, то оно делится по середине, согласно пунктуации
                if len(text) == 1:
                    text = split_text_by_punctuation(text[0], chunk_size)

            if isinstance(text, str):
                if filling_in_graph([Document(page_content=text)], paragraph_id_node, graph, models, True):
                    update_log(log_file, filename, prepare_title, title_id_node, prepare_paragraph, paragraph_id_node, processed_log, 'completed')
            elif isinstance(text, list):
                for text_small in text:
                    # Последняя попытка суммаризации большого предложения
                    if len(text_small) > chunk_size:
                        text_small = summary_text(text_small, chunk_size, summary_tokenizer, summary_model)
                    if filling_in_graph([Document(page_content=text_small)], paragraph_id_node, graph, models, True):
                        update_log(log_file, filename, prepare_title, title_id_node, prepare_paragraph, paragraph_id_node, processed_log, 'completed')
    
    break # Обработать один документ

# Закрываем соединение с графом
graph.close()

# 3 - Использование RAG графа

## 3.1 - Инициализация графа

In [ ]:
from langchain.graphs import Neo4jGraph
graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI"), 
    username=os.getenv("NEO4J_USERNAME"), 
    password=os.getenv("NEO4J_PASSWORD")
)

## 3.2 - Генерация запроса Cypher и получение ответа от LLM

In [ ]:
# Задаём вопрос
question = """
Чем является компьютерная атака?
""".strip()

In [ ]:
from langchain.chains import GraphCypherQAChain

### 3.2.1 - Использование GigaChat

In [ ]:
from langchain.chat_models.gigachat import GigaChat

In [ ]:
gigachat_api_key = getpass(prompt='Введите API ключ (Авторизационные данные) от GigaChat')

In [ ]:
model = GigaChat(credentials=gigachat_api_key, verify_ssl_certs=False, temperature=0.01)

cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm=model,
    qa_llm=model,
    graph=graph, 
    verbose=True,
    return_intermediate_steps=True
)

answer = cypher_chain.run(question)
print(answer)

### 3.2.2 - Использование LLM с HuhuggingFace

In [ ]:
for model in models:
    try:
        cypher_chain = GraphCypherQAChain.from_llm(
            cypher_llm=initialize_endpoint(model),
            qa_llm=initialize_endpoint(model),
            graph=graph, 
            verbose=True,
            return_intermediate_steps=True
        )
        # Задаём вопрос
        answer = cypher_chain.run(question)
        print(answer)
    except Exception as e:
        print(f"\nОшибка при использовании модели {model}: {e}\n")
        continue

## 3.2 - Использование векторного индекса

In [ ]:
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA

In [ ]:
vector_index = Neo4jVector.from_existing_graph(
    HuggingFaceEmbeddings(),
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
    index_name='tasks',
    node_label="Task",
    text_node_properties=['text', 'типы'],
    embedding_node_property='embedding',
)

# Поле экспериментальных способов получить внятный ответ от LLM

## Попытка сгенерировать запрос для трассировки от сущности до ключевого слова

In [ ]:
# Используем регулярные выражения для извлечения JSON-структуры
def extract_json(text):
    # Регулярное выражение для поиска JSON-объекта
    json_pattern = re.compile(r'\{.*\}', re.DOTALL)
    match = json_pattern.search(text)
    if match:
        return match.group(0)
    return None

In [ ]:
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
import json
from json_repair import repair_json

In [ ]:
# Шаблон для запроса
prompt_template = """User Instructions:
You will provide a query related to regulatory documents on information security. The LLM will extract relevant entities and keywords from your query and output a JSON containing these entities/keywords. The JSON can then be used to form a search query in a Neo4j graph database.

LLM Instructions:
You are provided with a user query related to regulatory documents on information security. Your task is to extract relevant entities and keywords from the query and output a JSON containing these entities/keywords.
Ensure both:
- entities
- keywords
are present in the JSON, even if they are empty arrays.

Additionally, make sure to include at least one entity and one keyword to trace the graph for finding an answer.

User Query:
{user_query}

LLM Response:"""

# Шаблон для вопроса и ответа
prompt = PromptTemplate(template=prompt_template, input_variables=["user_query"])

In [ ]:
all_entities = []
all_keywords = []

def split_and_trim(text):
    """
    Функция для разбивки текста на части и удаления половины символов у каждой части.
    """
    parts = text.split()
    parts = [part[:len(translate_to_rus(part))//2] for part in parts]
    return parts

for model in models:
    # Настройка модели
    llm = initialize_endpoint(model)
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    generated_text = llm_chain.invoke({"user_query": question})['text'].strip()
    print(generated_text)

    # Извлекаем JSON-объект из текста
    try:
        # Найти JSON в тексте
        json_match = re.search(r'\{.*?\}', generated_text, re.DOTALL)
        if json_match:
            json_text = json_match.group(0)
            extracted_json = json.loads(json_text)
            entities = extracted_json.get("entities", [])
            keywords = extracted_json.get("keywords", [])

            if entities:
                for entity in entities:
                    entity = lemmatize_word(entity.lower())
                    entity_parts = split_and_trim(entity)
                    for entity_part in entity_parts:
                        if entity_part not in all_entities:
                            all_entities.append(entity_part)

            if keywords:
                for keyword in keywords:
                    keyword = lemmatize_word(keyword.lower())
                    keyword_parts = split_and_trim(keyword)
                    for keyword_part in keyword_parts:
                        if keyword_part not in all_keywords:
                            all_keywords.append(keyword_part)
    except Exception as e:
        print(f"\nОшибка при использовании модели {model}: {e}\n")
        continue

# Удаление дубликатов
all_entities = list(set(all_entities))
all_keywords = list(set(all_keywords))

# Убираем дубликаты условий
unique_conditions = list(set(all_keywords) - set(all_entities))

# Формируем Cypher-запрос
entity_conditions = [f"toLower(start.text) CONTAINS toLower('{entity}')" for entity in all_entities]
keyword_conditions = [f"toLower(end.text) CONTAINS toLower('{keyword}')" for keyword in unique_conditions]

entity_conditions_str = " OR ".join(entity_conditions)
keyword_conditions_str = " OR ".join(keyword_conditions)

cypher_query = f"""
MATCH (start), (end)
WHERE ({entity_conditions_str}) AND ({keyword_conditions_str})
MATCH path = (start)-[*]->(end)
RETURN path
"""

print(cypher_query)